In [2]:
import cv2
import opencv_jupyter_ui as jcv2
import torch
from feat import Detector
from sklearn.svm import SVC
import pickle

import logging, sys
logging.disable(sys.maxsize)

detector = Detector(device='cuda' if torch.cuda.is_available() else 'cpu')

C:\Python310\lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
C:\Python310\lib\site-packages\feat\face_detectors\Retinaface\Retinaface_test.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_gl

In [3]:
emotion = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "sad": 4,
    "surprise": 5,
    "neutral": 6
}

In [4]:
with open('model_svc.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
def feeling(image, enable_aus=False, enable_emotion=False):
    face = detector.detect_faces(image)

    aus = None
    emotion_str = None

    if enable_emotion or enable_aus:
        landmark = detector.detect_landmarks(image, face)
        if enable_aus:
            aus = detector.detect_aus(image, landmark)
        if enable_emotion:
            emotion_str = model.predict(aus[0])[0]

    for (i, face) in enumerate(face[0]):
        f_hl = int(face[0])       # horizontal left
        f_vt = int(face[1])       # vertical top
        f_hr = int(face[2])       # horizontal right
        f_vb = int(face[3])       # vertical bottom

        if i == 0:
            color = (0, 255, 0)
        elif i == 1:
            color = (255, 0, 0)
        else:
            color = (0, 0, 255)

        thickness = 2
        cv2.line(image,(f_hl, f_vt), (f_hr, f_vt), color, thickness)   # top left to top right
        cv2.line(image,(f_hr, f_vt), (f_hr, f_vb), color, thickness)   # top right to bottom right
        cv2.line(image,(f_hr, f_vb), (f_hl, f_vb), color, thickness)   # bottom right to bottom left
        cv2.line(image,(f_hl, f_vb), (f_hl, f_vt), color, thickness)   # bottom left to top right

        if enable_emotion:
            cv2.putText(image, emotion_str, (f_hl, f_vt-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color)

    return image, aus, emotion

In [9]:
cam = cv2.VideoCapture(0)
counter = 0

while True:
    # check = True means we managed to get a frame.
    # If check = False, the device is not available, and we should quit.
    check, frame = cam.read()
    if not check:
        break

    new_frame, aus, emotion = feeling(frame, True, True)

    # OpenCV uses a separate window to display output.
    jcv2.imshow("video", new_frame)

    # Press ESC to exit.
    key = jcv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
jcv2.destroyAllWindows()

In [7]:
type(model)

sklearn.svm._classes.SVC